In [15]:
#import utileria as ut
#import arboles_numericos as an
import os
import random
import math
from collections import Counter
#import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt


In [16]:
def lee_csv(archivo, atributos=None, separador=','):
    """
    Lee un archivo CSV y regresa una lista de diccionarios.
    Se asume que la primera linea contiene el nombre de los atributos.

    Parámetros
    ----------
    archivo : str
        Nombre del archivo CSV.
    atributos : list(str)
        Lista de atributos a considerar. Si es None, se asume que la primera linea contiene los nombres de los atributos.
    separador : str
        Separador de columnas.
    """
    with open(archivo, 'r') as f:
        lineas = f.readlines()
    if atributos is None:
        columnas = lineas[0].strip().split(separador)
    else:
        columnas = atributos
    datos = []
    for l in lineas[1:]:
        datos.append({c: v for c, v in zip(columnas, l.strip().split(','))})
    return datos

#Arboles cualitativos

In [29]:
def entrena_arbol(datos, target, clase_default,
                  max_profundidad=None, acc_nodo=1, min_ejemplos=0):
    """
    Entrena un árbol de desición utilizando el criterio de entropía

    Parámetros:
    -----------
    datos: list(dict)
        Una lista de diccionarios donde cada diccionario representa una instancia.
        Cada diccionario tiene al menos un par llave-valor, donde la llave es el nombre de un atributo y el valor es el valor del atributo.
        Todos los diccionarios tienen la misma llave-valor.
    target: str
        El nombre del atributo que se quiere predecir
    clase_default: str
        El valor de la clase por default
    max_profundidad: int
        La máxima profundidad del árbol. Si es None, no hay límite de profundidad
    acc_nodo: int
        El porcentaje de acierto mínimo para considerar un nodo como hoja
    min_ejemplos: int
        El número mínimo de ejemplos para considerar un nodo como hoja

    Regresa:
    --------
    nodo: Nodo
        El nodo raíz del árbol de desición

    """
    atributos = list(datos[0].keys())
    atributos.remove(target)

    # Criterios para deterinar si es un nodo hoja
    if  len(datos) == 0 or len(atributos) == 0:
        return NodoQ(terminal=True, clase_default=clase_default)

    clases = Counter(d[target] for d in datos)
    clase_default = clases.most_common(1)[0][0]

    if (max_profundidad == 0 or
        len(datos) <= min_ejemplos or
        clases.most_common(1)[0][1] / len(datos) >= acc_nodo):

        return NodoQ(terminal=True, clase_default=clase_default)

    variable = selecciona_variable(datos, target, atributos)
    nodo = NodoQ(terminal=False, atributo=variable, clase_default=clase_default)

    for valor in set(d[variable] for d in datos):
        datos_hijo = [d for d in datos if d[variable] == valor]
        nodo.hijos[valor] = entrena_arbol(
            datos_hijo,
            target,
            clase_default,
            max_profundidad - 1 if max_profundidad is not None else None,
            acc_nodo, min_ejemplos
        )
    return nodo

In [30]:
def selecciona_variable(datos, target, atributos):
    """
    Selecciona el atributo que mejor separa las clases

    Parámetros:
    -----------
    datos: list(dict)
        Una lista de diccionarios donde cada diccionario representa una instancia.
        Cada diccionario tiene al menos un par llave-valor, donde la llave es el nombre de un atributo y el valor es el valor del atributo. Todos los diccionarios tienen la misma llave-valor.
    target: str
        El nombre del atributo que se quiere predecir
    atributos: list(str)
        La lista de atributos a considerar

    Regresa:
    --------
    atributo: str
        El nombre del atributo que mejor separa las clases
    """

    entropia = entropia_clase(datos, target)
    ganancia = {a: ganancia_informacion(datos, target, a, entropia) for a in atributos}
    return max(ganancia, key=ganancia.get)

In [31]:
def entropia_clase(datos, target):
    """
    Calcula la entropía de la clase

    Parámetros:
    -----------
    datos: list(dict)
        Una lista de diccionarios donde cada diccionario representa una instancia.
        Cada diccionario tiene al menos un par llave-valor, donde la llave es el nombre de un atributo y el valor es el valor del atributo. Todos los diccionarios tienen la misma llave-valor.
    target: str
        El nombre del atributo que se quiere predecir

    Regresa:
    --------
    entropia: float
        La entropía de la clase
    """

    clases = Counter(d[target] for d in datos)
    total = sum(clases.values())
    return -sum((c/total) * math.log2(c/total) for c in clases.values())

In [32]:
def ganancia_informacion(datos, target, atributo, entropia):
    """
    Calcula la ganancia de información de un atributo

    Parámetros:
    -----------
    datos: list(dict)
        Una lista de diccionarios donde cada diccionario representa una instancia.
        Cada diccionario tiene al menos un par llave-valor, donde la llave es el nombre de un atributo y el valor es el valor del atributo. Todos los diccionarios tienen la misma llave-valor.
    target: str
        El nombre del atributo que se quiere predecir
    atributo: str
        El nombre del atributo a considerar
    entropia: float
        La entropía de la clase

    Regresa:
    --------
    ganancia: float
        La ganancia de información del atributo
    """

    total = len(datos)
    ganancia = entropia

    for valor in set(d[atributo] for d in datos):
        datos_valor = [d for d in datos if d[atributo] == valor]
        ganancia -= (len(datos_valor) / total) * entropia_clase(datos_valor, target)
    return ganancia

In [34]:
def predice_arbol(arbol, datos):
    return [arbol.predice(d) for d in datos]

In [33]:
def evalua_arbol(arbol, datos, target):
    predicciones = predice_arbol(arbol, datos)
    return sum(1 for p, d in zip(predicciones, datos) if p == d[target]) / len(datos)

In [35]:
def imprime_arbol(nodo, nivel=0, valor=" "):
    if nodo.terminal:
        print("    " * nivel + f"Si valor es {valor}, la clase es {nodo.clase_default}")
    else:
        if valor == " ":
            print("    " * nivel
                  + f"Si el atributo es {nodo.atributo} entonces:")
        else:
            print("    " * nivel
                  + f"Si el valor es {valor} y el atributo es {nodo.atributo} entonces:")
        for valor, hijo in nodo.hijos.items():
            imprime_arbol(hijo, nivel + 1, valor)

In [36]:
class NodoQ:
    def __init__(self, terminal, clase_default, atributo=None):
        self.terminal = terminal
        self.clase_default = clase_default
        self.atributo = atributo
        self.hijos = {}

    def predice(self, instancia):
        if self.terminal:
            return self.clase_default
        valor = instancia[self.atributo]
        if valor not in self.hijos:
            return self.clase_default
        return self.hijos[valor].predice(instancia)

In [37]:
def main():
    datos = [
        {"color": "rojo", "tamano": "grande", "sabor": "dulce", "clase": "manzana"},
        {"color": "verde", "tamano": "grande", "sabor": "dulce", "clase": "sandia"},
        {"color": "rojo", "tamano": "pequeno", "sabor": "dulce", "clase": "uva"},
        {"color": "verde", "tamano": "grande", "sabor": "amargo", "clase": "sandia"},
        {"color": "verde", "tamano": "pequeno", "sabor": "amargo", "clase": "uva"},
        {"color": "rojo", "tamano": "grande", "sabor": "amargo", "clase": "manzana"},
        {"color": "rojo", "tamano": "pequeno", "sabor": "dulce", "clase": "uva"},
        {"color": "verde", "tamano": "pequeno", "sabor": "dulce", "clase": "uva"},
        {"color": "rojo", "tamano": "grande", "sabor": "amargo", "clase": "manzana"},
        {"color": "verde", "tamano": "pequeno", "sabor": "amargo", "clase": "uva"},
        {"color": "rojo", "tamano": "pequeno", "sabor": "amargo", "clase": "manzana"},
        {"color": "verde", "tamano": "grande", "sabor": "dulce", "clase": "sandia"},
        {"color": "rojo", "tamano": "pequeno", "sabor": "dulce", "clase": "uva"},
        {"color": "verde", "tamano": "pequeno", "sabor": "amargo", "clase": "uva"},
        {"color": "rojo", "tamano": "grande", "sabor": "amargo", "clase": "manzana"},
        {"color": "verde", "tamano": "pequeno", "sabor": "dulce", "clase": "uva"},
        {"color": "rojo", "tamano": "grande", "sabor": "amargo", "clase": "manzana"}
    ]

    raiz = entrena_arbol(datos, "clase", "uva")
    imprime_arbol(raiz)

    acc = evalua_arbol(raiz, datos, "clase")
    print(f"El acierto en los mismos datos que se entrenó es {acc}")
    return None

In [38]:
if __name__ == "__main__":
    main()

Si el atributo es tamano entonces:
    Si el valor es pequeno y el atributo es color entonces:
        Si el valor es rojo y el atributo es sabor entonces:
            Si valor es amargo, la clase es manzana
            Si valor es dulce, la clase es uva
        Si valor es verde, la clase es uva
    Si el valor es grande y el atributo es color entonces:
        Si valor es rojo, la clase es manzana
        Si valor es verde, la clase es sandia
El acierto en los mismos datos que se entrenó es 1.0


#arboles numericos

In [17]:
def entrena_arbol(datos, target, clase_default,
                  max_profundidad=None, acc_nodo=1.0, min_ejemplos=0,
                  variables_seleccionadas=None):
    """
    Entrena un árbol de desición utilizando el criterio de entropía

    Parámetros:
    -----------
    datos: list(dict)
        Una lista de diccionarios donde cada diccionario representa una instancia.
        Cada diccionario tiene al menos un par llave-valor, donde la llave es el nombre de un atributo y el valor es el valor del atributo.
        Todos los diccionarios tienen la misma llave-valor.
    target: str
        El nombre del atributo que se quiere predecir
    clase_default: str
        El valor de la clase por default
    max_profundidad: int
        La máxima profundidad del árbol. Si es None, no hay límite de profundidad
    acc_nodo: int
        El porcentaje de acierto mínimo para considerar un nodo como hoja
    min_ejemplos: int
        El número mínimo de ejemplos para considerar un nodo como hoja
    variables_seleccionadas: list(str)
        Lista de variables a considerar. Si es None, se consideran todas las variables, esto apica para árboles aleagtorios y lo tendrán que implementar en la tarea.

    Regresa:
    --------
    nodo: Nodo
        El nodo raíz del árbol de desición

    """
    atributos = list(datos[0].keys())
    atributos.remove(target)

    # Criterios para deterinar si es un nodo hoja
    if  len(datos) == 0 or len(atributos) == 0:
        return NodoN(terminal=True, clase_default=clase_default)

    clases = Counter(d[target] for d in datos)
    clase_default = clases.most_common(1)[0][0]

    if (max_profundidad == 0 or
        len(datos) <= min_ejemplos or
        clases.most_common(1)[0][1] / len(datos) >= acc_nodo):

        return NodoN(terminal=True, clase_default=clase_default)

    variable, valor = selecciona_variable_valor(
        datos, target, atributos
    )
    nodo = NodoN(
        terminal=False,
        clase_default=clase_default,
        atributo=variable,
        valor=valor
    )
    nodo.hijo_menor = entrena_arbol(
        [d for d in datos if d[variable] < valor],
        target,
        clase_default,
        max_profundidad - 1 if max_profundidad is not None else None,
        acc_nodo, min_ejemplos, variables_seleccionadas
    )
    nodo.hijo_mayor = entrena_arbol(
        [d for d in datos if d[variable] >= valor],
        target,
        clase_default,
        max_profundidad - 1 if max_profundidad is not None else None,
        acc_nodo, min_ejemplos, variables_seleccionadas
    )
    return nodo

In [18]:
def selecciona_variable_valor(datos, target, atributos):
    """
    Selecciona el atributo y el valor que mejor separa las clases

    Parámetros:
    -----------
    datos: list(dict)
        Una lista de diccionarios donde cada diccionario representa una instancia.
        Cada diccionario tiene al menos un par llave-valor, donde la llave es el nombre de un atributo y el valor es el valor del atributo. Todos los diccionarios tienen la misma llave-valor.
    target: str
        El nombre del atributo que se quiere predecir
    atributos: list(str)
        La lista de atributos a considerar

    Regresa:
    --------
    atributo: str
        El nombre del atributo que mejor separa las clases
    valor: float
        El valor del atributo que mejor separa las clases
    """

    entropia = entropia_clase(datos, target)
    mejor = max(
        ((a, maxima_ganancia_informacion(datos, target, a, entropia))
            for a in atributos),
        key=lambda x: x[1][1]
    )
    return mejor[0], mejor[1][0]

In [19]:
def entropia_clase(datos, target):
    """
    Calcula la entropía de la clase

    Parámetros:
    -----------
    datos: list(dict)
        Una lista de diccionarios donde cada diccionario representa una instancia.
        Cada diccionario tiene al menos un par llave-valor, donde la llave es el nombre de un atributo y el valor es el valor del atributo. Todos los diccionarios tienen la misma llave-valor.
    target: str
        El nombre del atributo que se quiere predecir

    Regresa:
    --------
    entropia: float
        La entropía de la clase
    """

    clases = Counter(d[target] for d in datos)
    total = sum(clases.values())
    return -sum((c/total) * math.log2(c/total) for c in clases.values())


In [20]:
def maxima_ganancia_informacion(datos, target, atributo, entropia):
    """
    Calcula la ganancia de información de un atributo

    Parámetros:
    -----------
    datos: list(dict)
        Una lista de diccionarios donde cada diccionario representa una instancia.
        Cada diccionario tiene al menos un par llave-valor, donde la llave es el nombre de un atributo y el valor es el valor del atributo. Todos los diccionarios tienen la misma llave-valor.
    target: str
        El nombre del atributo que se quiere predecir
    atributo: str
        El nombre del atributo a considerar
    entropia: float
        La entropía de la clase

    Regresa:
    --------
    valor: float
        El valor del atributo que mejor separa las clases
    ganancia: float
        La ganancia de información del atributo dividiendo en ese valor

    """

    lista_valores = [(d[atributo], d[target]) for d in datos]
    lista_valores.sort(key=lambda x: x[0])
    lista_valor_ganancia = []
    for (v1, v2) in zip(lista_valores[:-1], lista_valores[1:]):
        if v1[1] != v2[1]:
            valor = (v1[0] + v2[0]) / 2
            ganancia = ganancia_informacion(datos, target, atributo, valor, entropia)
            lista_valor_ganancia.append((valor, ganancia))
    return max(lista_valor_ganancia, key=lambda x: x[1])

In [21]:
def ganancia_informacion(datos, target, atributo, valor, entropia):
    """
    Calcula la ganancia de información de un atributo dividiendo en un valor

    Parámetros:
    -----------
    datos: list(dict)
        Una lista de diccionarios donde cada diccionario representa una instancia.
        Cada diccionario tiene al menos un par llave-valor, donde la llave es el nombre de un atributo y el valor es el valor del atributo. Todos los diccionarios tienen la misma llave-valor.
    target: str
        El nombre del atributo que se quiere predecir
    atributo: str
        El nombre del atributo a considerar
    valor: float
        El valor del atributo a considerar
    entropia: float
        La entropía de la clase

    Regresa:
    --------
    ganancia: float
        La ganancia de información del atributo dividiendo en ese valor
    """

    datos_menor = [d for d in datos if d[atributo] < valor]
    datos_mayor = [d for d in datos if d[atributo] >= valor]

    entropia_menor = entropia_clase(datos_menor, target)
    entropia_mayor = entropia_clase(datos_mayor, target)

    total = len(datos)
    total_menor = len(datos_menor)
    total_mayor = len(datos_mayor)

    return (
        entropia
        - (total_menor / total) * entropia_menor
        - (total_mayor / total) * entropia_mayor
    )

In [22]:
def predice_arbol(arbol, datos):
    return [arbol.predice(d) for d in datos]

In [23]:
def evalua_arbol(arbol, datos, target):
    predicciones = predice_arbol(arbol, datos)
    return sum(1 for p, d in zip(predicciones, datos) if p == d[target]) / len(datos)

In [24]:
def imprime_arbol(nodo, nivel=0):
    if nodo.terminal:
        print("    " * nivel + f"La clase es {nodo.clase_default}")
    else:
        print("    " * nivel + f"Si {nodo.atributo} < {nodo.valor} entonces:")
        imprime_arbol(nodo.hijo_menor, nivel + 1)
        print("    " * nivel + f"Si {nodo.atributo} >= {nodo.valor} entonces:")
        imprime_arbol(nodo.hijo_mayor, nivel + 1)

In [25]:
class NodoN:
    def __init__(self, terminal, clase_default, atributo=None, valor=None):
        self.terminal = terminal
        self.clase_default = clase_default
        self.atributo = atributo
        self.valor = valor
        self.hijo_menor = None
        self.hijo_mayor = None

    def predice(self, instancia):
        if self.terminal:
            return self.clase_default
        if instancia[self.atributo] < self.valor:
            return self.hijo_menor.predice(instancia)
        return self.hijo_mayor.predice(instancia)

In [26]:
def main():
    datos = [
        {"atributo1": 1, "atributo2": 1, "clase": "positiva"},
        {"atributo1": 2, "atributo2": 1, "clase": "positiva"},
        {"atributo1": 3, "atributo2": 1, "clase": "positiva"},
        {"atributo1": 4, "atributo2": 1, "clase": "positiva"},
        {"atributo1": 1, "atributo2": 2, "clase": "positiva"},
        {"atributo1": 2, "atributo2": 2, "clase": "positiva"},
        {"atributo1": 3, "atributo2": 2, "clase": "positiva"},
        {"atributo1": 4, "atributo2": 2, "clase": "positiva"},
        {"atributo1": 1, "atributo2": 3, "clase": "negativa"},
        {"atributo1": 2, "atributo2": 3, "clase": "negativa"},
        {"atributo1": 3, "atributo2": 3, "clase": "negativa"},
        {"atributo1": 4, "atributo2": 3, "clase": "negativa"},
        {"atributo1": 1, "atributo2": 4, "clase": "positiva"},
        {"atributo1": 2, "atributo2": 4, "clase": "positiva"},
        {"atributo1": 3, "atributo2": 4, "clase": "positiva"},
        {"atributo1": 4, "atributo2": 4, "clase": "positiva"},

   ]

    raiz = entrena_arbol(datos, "clase", "positiva")
    imprime_arbol(raiz)

    acc = evalua_arbol(raiz, datos, "clase")
    print(f"El acierto en los mismos datos que se entrenó es {acc}")
    return None

In [28]:
if __name__ == "__main__":
    main()

Si atributo2 < 2.5 entonces:
    La clase es positiva
Si atributo2 >= 2.5 entonces:
    Si atributo2 < 3.5 entonces:
        La clase es negativa
    Si atributo2 >= 3.5 entonces:
        La clase es positiva
El acierto en los mismos datos que se entrenó es 1.0
